In [ ]:
# STEP 1: Import required libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when


In [ ]:
spark = SparkSession.builder.appName("DiabetesDataCleaningRequiredOnly").getOrCreate()

In [ ]:
df = spark.read.csv("modified_diabetic_data.csv", header=True, inferSchema=True)

In [ ]:
# Step 2: Select only the required columns
required_columns = [
    "race", "gender", "age",  # Patient Demographics
    "diag_1", "diag_2", "diag_3",  # Diagnosis Codes
    "time_in_hospital",  # Length of Stay
    "num_procedures", "num_lab_procedures", "num_medications",  # Treatment Procedures
    "readmitted",  # Readmission Rates
    "cost"  # Cost of Care
]
df_cleaned = df.select(required_columns)

In [ ]:
# Step 3: Handle missing values in the selected columns
# Replace '?' with appropriate values
most_frequent_race = df_cleaned.groupBy("race").count().orderBy(col("count").desc()).first()["race"]
df_cleaned = df_cleaned.withColumn("race", when(col("race") == "?", most_frequent_race).otherwise(col("race")))
df_cleaned = df_cleaned.withColumn("gender", when(col("gender") == "?", "Unknown").otherwise(col("gender")))
df_cleaned = df_cleaned.withColumn("gender", when(col("gender") == "Unknown/Invalid", "Unknown").otherwise(col("gender")))
df_cleaned = df_cleaned.withColumn("diag_1", when(col("diag_1") == "?", "Unknown").otherwise(col("diag_1")))
df_cleaned = df_cleaned.withColumn("diag_2", when(col("diag_2") == "?", "Unknown").otherwise(col("diag_2")))
df_cleaned = df_cleaned.withColumn("diag_3", when(col("diag_3") == "?", "Unknown").otherwise(col("diag_3")))


In [ ]:
df_cleaned.show(10, truncate=False)

+---------------+------+--------+------+-------+-------+----------------+--------------+------------------+---------------+----------+------------------+
|race           |gender|age     |diag_1|diag_2 |diag_3 |time_in_hospital|num_procedures|num_lab_procedures|num_medications|readmitted|cost              |
+---------------+------+--------+------+-------+-------+----------------+--------------+------------------+---------------+----------+------------------+
|Caucasian      |Female|[0-10)  |250.83|Unknown|Unknown|1               |0             |41                |1              |No        |5385.64144960542  |
|Caucasian      |Female|[10-20) |276   |250.01 |255    |3               |0             |59                |18             |Yes       |8960.53495702278  |
|AfricanAmerican|Female|[20-30) |648   |250    |V27    |2               |5             |11                |13             |No        |8717.027146672363 |
|Caucasian      |Male  |[30-40) |8     |250.43 |403    |2               |1  

In [ ]:
# Step 5: Save the cleaned dataset
pandas_df_cleaned = df_cleaned.toPandas()
cleaned_file_path = "cleaned_diabetic_data_required_only.csv"
pandas_df_cleaned.to_csv(cleaned_file_path)

print(f"\nCleaned dataset (required columns only) saved successfully at: {cleaned_file_path}")


Cleaned dataset (required columns only) saved successfully at: cleaned_diabetic_data_required_only.csv


Data Transformation


In [ ]:
df = spark.read.csv("cleaned_diabetic_data_required_only.csv", header=True, inferSchema=True)

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, regexp_replace # Import regexp_replace
from pyspark.sql.types import IntegerType # Import IntegerType
# Extract the lower and upper bounds from the age range (e.g., [0-10) -> 0 and 10)
df = df.withColumn("age_lower", regexp_replace(col("age"), r"\[(\d+)-(\d+)\)", "$1").cast(IntegerType()))
df = df.withColumn("age_upper", regexp_replace(col("age"), r"\[(\d+)-(\d+)\)", "$2").cast(IntegerType()))

# Compute the midpoint for the age range
df = df.withColumn("age", ((col("age_lower") + col("age_upper")) / 2).cast(IntegerType()))

# Drop the temporary columns
df = df.drop("age_lower", "age_upper")

# Show the transformed age column
df.select("age").show(5)

+---+
|age|
+---+
|  5|
| 15|
| 25|
| 35|
| 45|
+---+
only showing top 5 rows



In [ ]:
from pyspark.sql.types import IntegerType, FloatType
# Cast numerical columns to appropriate types
df = df.withColumn("time_in_hospital", col("time_in_hospital").cast(IntegerType()))
df = df.withColumn("num_procedures", col("num_procedures").cast(IntegerType()))
df = df.withColumn("num_lab_procedures", col("num_lab_procedures").cast(IntegerType()))
df = df.withColumn("num_medications", col("num_medications").cast(IntegerType()))
df = df.withColumn("cost", col("cost").cast(FloatType()))

# Verify the schema
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- race: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- diag_1: string (nullable = true)
 |-- diag_2: string (nullable = true)
 |-- diag_3: string (nullable = true)
 |-- time_in_hospital: integer (nullable = true)
 |-- num_procedures: integer (nullable = true)
 |-- num_lab_procedures: integer (nullable = true)
 |-- num_medications: integer (nullable = true)
 |-- readmitted: string (nullable = true)
 |-- cost: float (nullable = true)



In [ ]:
# Calculate IQR for cost
quantiles = df.approxQuantile("cost", [0.25, 0.75], 0.05)
q1, q3 = quantiles[0], quantiles[1]
iqr = q3 - q1
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr

# Filter out outliers in cost
df = df.filter((col("cost") >= lower_bound) & (col("cost") <= upper_bound))

# Show the filtered dataset
df.select("cost").describe().show()

+-------+-----------------+
|summary|             cost|
+-------+-----------------+
|  count|            98424|
|   mean|8523.455546217469|
| stddev|3112.769478960883|
|    min|        1441.4276|
|    max|        16450.281|
+-------+-----------------+



Feature engineering

In [ ]:
df = spark.read.csv("cleaned_diabetic_data_required_only.csv", header=True, inferSchema=True)

In [ ]:
# Create a feature for total diagnoses
df = df.withColumn("total_diagnoses",
                   when(col("diag_1") != "Unknown", 1).otherwise(0) +
                   when(col("diag_2") != "Unknown", 1).otherwise(0) +
                   when(col("diag_3") != "Unknown", 1).otherwise(0))

# Show the new column
df.select("diag_1", "diag_2", "diag_3", "total_diagnoses").show(5)

+------+-------+-------+---------------+
|diag_1| diag_2| diag_3|total_diagnoses|
+------+-------+-------+---------------+
|250.83|Unknown|Unknown|              1|
|   276| 250.01|    255|              3|
|   648|    250|    V27|              3|
|     8| 250.43|    403|              3|
|   197|    157|    250|              3|
+------+-------+-------+---------------+
only showing top 5 rows



In [ ]:
# Create cost_per_day feature
df = df.withColumn("cost_per_day", col("cost") / col("time_in_hospital"))

# Handle division by zero
df = df.withColumn("cost_per_day", when(col("time_in_hospital") == 0, 0).otherwise(col("cost_per_day")))

# Show the new column
df.select("cost", "time_in_hospital", "cost_per_day").show(5)

+---------+----------------+------------------+
|     cost|time_in_hospital|      cost_per_day|
+---------+----------------+------------------+
|5385.6416|               1|   5385.6416015625|
| 8960.535|               3|2986.8450520833335|
| 8717.027|               2|    4358.513671875|
|7204.9355|               2|   3602.4677734375|
|7023.0815|               1|  7023.08154296875|
+---------+----------------+------------------+
only showing top 5 rows



In [ ]:
# Create total_procedures feature
df = df.withColumn("total_procedures", col("num_procedures") + col("num_lab_procedures"))

# Show the new column
df.select("num_procedures", "num_lab_procedures", "total_procedures").show(5)

+--------------+------------------+----------------+
|num_procedures|num_lab_procedures|total_procedures|
+--------------+------------------+----------------+
|             0|                41|              41|
|             0|                59|              59|
|             5|                11|              16|
|             1|                44|              45|
|             0|                51|              51|
+--------------+------------------+----------------+
only showing top 5 rows



In [ ]:
# Create age bins
df = df.withColumn("age_group",
                   when(col("age") < 18, "Child")
                   .when((col("age") >= 18) & (col("age") < 35), "Young Adult")
                   .when((col("age") >= 35) & (col("age") < 65), "Adult")
                   .otherwise("Senior"))

# Show the new column
df.select("age", "age_group").show(5)

+---+-----------+
|age|  age_group|
+---+-----------+
|  5|      Child|
| 15|      Child|
| 25|Young Adult|
| 35|      Adult|
| 45|      Adult|
+---+-----------+
only showing top 5 rows



In [ ]:
from pyspark.ml.feature import StringIndexer

# Convert readmitted to a binary index
indexer = StringIndexer(inputCol="readmitted", outputCol="readmitted_index")
df = indexer.fit(df).transform(df)

# Show the new column
df.select("readmitted", "readmitted_index").show(5)

+----------+----------------+
|readmitted|readmitted_index|
+----------+----------------+
|        No|             0.0|
|       Yes|             1.0|
|        No|             0.0|
|        No|             0.0|
|        No|             0.0|
+----------+----------------+
only showing top 5 rows



In [ ]:
from pyspark.sql.functions import avg # Import the avg function

# Calculate the average cost
avg_cost = df.select(avg("cost").alias("avg_cost")).collect()[0]["avg_cost"]

# Create high_cost indicator
df = df.withColumn("high_cost", when(col("cost") > avg_cost, 1).otherwise(0))

# Show the new column
df.select("cost", "high_cost").show(5)

+---------+---------+
|     cost|high_cost|
+---------+---------+
|5385.6416|        0|
| 8960.535|        1|
| 8717.027|        1|
|7204.9355|        0|
|7023.0815|        0|
+---------+---------+
only showing top 5 rows



In [ ]:
# Create diagnosis_complexity feature
df = df.withColumn("diagnosis_complexity", when(col("total_diagnoses") > 1, 1).otherwise(0))

# Show the new column
df.select("total_diagnoses", "diagnosis_complexity").show(5)

+---------------+--------------------+
|total_diagnoses|diagnosis_complexity|
+---------------+--------------------+
|              1|                   0|
|              3|                   1|
|              3|                   1|
|              3|                   1|
|              3|                   1|
+---------------+--------------------+
only showing top 5 rows



In [ ]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml import Pipeline

# Index and encode race and gender
race_indexer = StringIndexer(inputCol="race", outputCol="race_index")
gender_indexer = StringIndexer(inputCol="gender", outputCol="gender_index")
race_encoder = OneHotEncoder(inputCols=["race_index"], outputCols=["race_encoded"])
gender_encoder = OneHotEncoder(inputCols=["gender_index"], outputCols=["gender_encoded"])

# Create a pipeline for encoding
pipeline = Pipeline(stages=[race_indexer, gender_indexer, race_encoder, gender_encoder])
df = pipeline.fit(df).transform(df)

# Drop intermediate columns
df = df.drop("race_index", "gender_index")

# Show the encoded columns
df.select("race", "race_encoded", "gender", "gender_encoded").show(5)

+---------------+-------------+------+--------------+
|           race| race_encoded|gender|gender_encoded|
+---------------+-------------+------+--------------+
|      Caucasian|(4,[0],[1.0])|Female| (2,[0],[1.0])|
|      Caucasian|(4,[0],[1.0])|Female| (2,[0],[1.0])|
|AfricanAmerican|(4,[1],[1.0])|Female| (2,[0],[1.0])|
|      Caucasian|(4,[0],[1.0])|  Male| (2,[1],[1.0])|
|      Caucasian|(4,[0],[1.0])|  Male| (2,[1],[1.0])|
+---------------+-------------+------+--------------+
only showing top 5 rows



In [ ]:
# Create procedure_intensity feature based on the total_procedures
# Define a threshold for high intensity (you can adjust this)
high_intensity_threshold = 5

# Create a new column called procedure_intensity
df = df.withColumn("procedure_intensity", when(col("total_procedures") > high_intensity_threshold, 1).otherwise(0))

# Now, re-run the select statement to include procedure_intensity:
df.select("age", "age_group", "total_diagnoses", "diagnosis_complexity",
          "total_procedures", "procedure_intensity", "cost", "cost_per_day",
          "high_cost", "readmitted", "readmitted_index", "race_encoded", "gender_encoded").show(5)

+---+-----------+---------------+--------------------+----------------+-------------------+---------+------------------+---------+----------+----------------+-------------+--------------+
|age|  age_group|total_diagnoses|diagnosis_complexity|total_procedures|procedure_intensity|     cost|      cost_per_day|high_cost|readmitted|readmitted_index| race_encoded|gender_encoded|
+---+-----------+---------------+--------------------+----------------+-------------------+---------+------------------+---------+----------+----------------+-------------+--------------+
|  5|      Child|              1|                   0|              41|                  1|5385.6416|   5385.6416015625|        0|        No|             0.0|(4,[0],[1.0])| (2,[0],[1.0])|
| 15|      Child|              3|                   1|              59|                  1| 8960.535|2986.8450520833335|        1|       Yes|             1.0|(4,[0],[1.0])| (2,[0],[1.0])|
| 25|Young Adult|              3|                   1|      

In [ ]:
# Convert to Pandas and save as CSV
df_pandas = df.toPandas()
df_pandas.to_csv("transformed_engineered_healthcare_dataset.csv", index=False)

# Download the file
from google.colab import files
files.download("transformed_engineered_healthcare_dataset.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>